In [1]:
# Loading libraries
%matplotlib inline
import os
import os.path
from netCDF4 import Dataset, num2date, date2num
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc 
import fnmatch
from cycler import cycler
import pandas as pd
from pandas import read_csv, to_datetime, DataFrame
import datetime
import time
import matplotlib.dates as mdates
from datetime import datetime, timedelta

from matplotlib.path import Path
from matplotlib.patches import PathPatch
from mpl_toolkits.basemap import Basemap # Basemap pour les cartes

In [30]:
# Chemin des fichiers à utiliser
file_path = './ANNUAL_MEANS_BestAjustRivE_BoostW_BOUCLE_NEW/TEM/'
full_list = os.listdir(file_path)
full_list.sort()
full_list

['moyenne_tem_1970.nc',
 'moyenne_tem_1971.nc',
 'moyenne_tem_1972.nc',
 'moyenne_tem_1973.nc',
 'moyenne_tem_1974.nc',
 'moyenne_tem_1975.nc',
 'moyenne_tem_1976.nc',
 'moyenne_tem_1977.nc',
 'moyenne_tem_1978.nc',
 'moyenne_tem_1979.nc',
 'moyenne_tem_1980.nc',
 'moyenne_tem_1981.nc',
 'moyenne_tem_1982.nc',
 'moyenne_tem_1983.nc',
 'moyenne_tem_1984.nc',
 'moyenne_tem_1985.nc',
 'moyenne_tem_1986.nc',
 'moyenne_tem_1987.nc',
 'moyenne_tem_1988.nc',
 'moyenne_tem_1989.nc',
 'moyenne_tem_1990.nc',
 'moyenne_tem_1991.nc',
 'moyenne_tem_1992.nc',
 'moyenne_tem_1993.nc',
 'moyenne_tem_1994.nc',
 'moyenne_tem_1995.nc',
 'moyenne_tem_1996.nc',
 'moyenne_tem_1997.nc',
 'moyenne_tem_1998.nc',
 'moyenne_tem_1999.nc',
 'moyenne_tem_2000.nc',
 'moyenne_tem_2001.nc',
 'moyenne_tem_2002.nc',
 'moyenne_tem_2003.nc',
 'moyenne_tem_2004.nc',
 'moyenne_tem_2005.nc',
 'moyenne_tem_2006.nc',
 'moyenne_tem_2007.nc',
 'moyenne_tem_2008.nc',
 'moyenne_tem_2009.nc',
 'moyenne_tem_2010.nc',
 'moyenne_tem_20

In [3]:
# LOADING GRID FILE AND VARIABLES
!pwd
grid_file = '../DATA/grille.nc' 
print 'file exist:', os.path.isfile(grid_file)

f = Dataset(grid_file) # open netcdf
# extracting variables
lon_t = f.variables['longitude_t'][:] # extracting lon data
lat_t = f.variables['latitude_t'][:] # extracting lat data
#depth_t = f.variables['depth_t'][:]
dz_t = f.variables['dz_t'][:]
bathy = f.variables['h_w'][:]
print 'Bathy :', np.shape(bathy), 'max :', np.max(bathy), 'min :', np.min(bathy)

f.close()

new_bath = np.ma.masked_where(bathy<200, bathy)

/home/dosa/MON_ENV_PYTHON/LAST_VALID
file exist: True
Bathy : (160, 394) max : 5054.49 min : 20.5226


In [24]:
# NEW VERSION

# For 196 m
count = 0
time_s = np.zeros([len(full_list)]) # replace file_list here

for file in full_list:
    print file
    
    # extracting variables
    f = Dataset(file_path+file) # open netcdf
    chl_tot_196 = f.variables['phosphate'][0,:,:,:] # extracting # [0,30:43,:,:]
    f.close()
    
    # Creating empty arrays to store integrated values
    vars()['chl_tot_196_'+str(file[0:4])] = np.zeros([160,394])
    
    # At each lvl
    for z in np.arange(29,43): # 200m
    #for z in np.arange(31,43): # 150m
        # taking this particular lvl, masking it to avoid useless computations
        vars()['chl_tot_196_lvl_'+str(z)] = np.ma.masked_where(bathy<200, chl_tot_196[z,:,:])
            
    # for each (2D) point
    for i in np.arange(0,160):
        for j in np.arange(0,394):
            if np.ma.is_masked(new_bath[i,j]) == False: # to avoid useless points
                chl_tot_ij = 0
                chl_tot_this_lvl = 0

                for z in np.arange(29,43): # there we need to consider one less lvl
                    chl_tot_this_lvl = vars()['chl_tot_196_lvl_'+str(z)][i,j] * dz_t[z,i,j]
                    chl_tot_ij = np.nansum([chl_tot_ij, chl_tot_this_lvl])

                # now putting our int chl value in the output array
                vars()['chl_tot_196_'+str(file[0:4])][i,j] = chl_tot_ij
                
    print 'Creating ncdf'
    ### Creating NetCDF files for our outputs! ###
    mask_alex = Dataset('./NC_INT_OUTPUTS_ALEX/Pho_Int200_'+str(file[0:4])+'.nc', 'w', format='NETCDF4_CLASSIC')
    # creating dimensions
    
    # A REMODIFIER TOUT ÇA !!!
    ni_t = mask_alex.createDimension('ni_t', 160) 
    # A REMODIFIER TOUT ÇA !!!
    nj_t = mask_alex.createDimension('nj_t', 394) 
    # creating variable with dimensions
    mask_t = mask_alex.createVariable('Pho_Int_0-212', np.int32, ('ni_t', 'nj_t')) # A REMODIFIER TOUT ÇA !!!
    mask_t[:] =vars()['chl_tot_196_'+str(file[0:4])][:] # putting our df inside our variable
    mask_alex.close() # writing the netcdf file!
                               
    #print ' '

1970.nc
Creating ncdf
1971.nc
Creating ncdf
1972.nc
Creating ncdf
1973.nc
Creating ncdf
1974.nc
Creating ncdf
1975.nc
Creating ncdf
1976.nc
Creating ncdf
1977.nc
Creating ncdf
1978.nc
Creating ncdf
1979.nc
Creating ncdf
1980.nc
Creating ncdf
1981.nc
Creating ncdf
1982.nc
Creating ncdf
1983.nc
Creating ncdf
1984.nc
Creating ncdf
1985.nc
Creating ncdf
1986.nc
Creating ncdf
1987.nc
Creating ncdf
1988.nc
Creating ncdf
1989.nc
Creating ncdf
1990.nc
Creating ncdf
1991.nc
Creating ncdf
1992.nc
Creating ncdf
1993.nc
Creating ncdf
1994.nc
Creating ncdf
1995.nc
Creating ncdf
1996.nc
Creating ncdf
1997.nc
Creating ncdf
1998.nc
Creating ncdf
1999.nc
Creating ncdf
2000.nc
Creating ncdf
2001.nc
Creating ncdf
2002.nc
Creating ncdf
2003.nc
Creating ncdf
2004.nc
Creating ncdf
2005.nc
Creating ncdf
2006.nc
Creating ncdf
2007.nc
Creating ncdf
2008.nc
Creating ncdf
2009.nc
Creating ncdf
2010.nc
Creating ncdf
2011.nc
Creating ncdf


In [35]:
# NEW TENTATIVE TEM pond + ecriture ncdf

# For 196 m
count = 0
time_s = np.zeros([len(full_list)]) # replace file_list here

for file in full_list:
    print file
    
    # extracting variables
    f = Dataset(file_path+file) # open netcdf
    chl_tot_196 = f.variables['tem'][0,:,:,:] # extracting # [0,30:43,:,:]
    f.close()
    
    # Creating empty arrays to store integrated values
    vars()['chl_tot_196_'+str(file[12:16])] = np.zeros([160,394])
    vars()['chl_tot_196_'+str(file[12:16])][:] = np.nan
    
    # At each lvl
    for z in np.arange(31,43):
        # taking this particular lvl, masking it to avoid useless computations
        vars()['chl_tot_196_lvl_'+str(z)] = np.ma.masked_where(bathy<200, chl_tot_196[z,:,:])
            
    # for each (2D) point
    for i in np.arange(0,160):
        for j in np.arange(0,394):
            if np.ma.is_masked(new_bath[i,j]) == False: # to avoid useless points
                chl_tot_ij = 0
                chl_tot_this_lvl = 0

                #for z in np.arange(29,43): # 200m
                for z in np.arange(31,43): # 150m
                    chl_tot_this_lvl = vars()['chl_tot_196_lvl_'+str(z)][i,j] * dz_t[z,i,j]
                    chl_tot_ij = np.nansum([chl_tot_ij, chl_tot_this_lvl])
                    
                # TEM => weighing by depth
                chl_tot_ij = chl_tot_ij/150.96777 # for 151m case
                # 211.94522 # for 212m case

                # now putting our int chl value in the output array
                vars()['chl_tot_196_'+str(file[12:16])][i,j] = chl_tot_ij
                
    print 'Creating ncdf'
    ### Creating NetCDF files for our outputs! ###
    dates = []
    
    mask_alex = Dataset('./NC_INT_OUTPUTS_ALEX/Tem_Int150_'+str(file[12:16])+'.nc', 'w', format='NETCDF4_CLASSIC')
    # creating dimensions
    time = mask_alex.createDimension('time',1)
    #ni_t = mask_alex.createDimension('ni_t', 160) # save 11.04
    #nj_t = mask_alex.createDimension('nj_t', 394) # save 11.04
    ni_t = mask_alex.createDimension('nj_t', 160) 
    nj_t = mask_alex.createDimension('ni_t', 394)
    
    # creating variable with dimensions
    times = mask_alex.createVariable('time', np.float64, ('time',))  # float64
    times.units = 'seconds since 1970-06-15 12:10:01' 
    times.calendar = 'gregorian'
    
    dates.append(datetime(int(file[12:16]), 10, 3, 12, 10, 1)) # according to ncdf files already existing 
    times[:] = date2num(dates, units = times.units, calendar = times.calendar)
    #times[:] = date2num(dates)
    #print dates

    #mask_t = mask_alex.createVariable('Tem_Int_0-151', np.float32, ('time', 'ni_t', 'nj_t'), fill_value=-9999) # save 11.04
    mask_t = mask_alex.createVariable('Tem_Int_0-151', np.float32, ('time', 'nj_t', 'ni_t'), fill_value=-9999) # 11.04
    #mask_t.units = 'mmol.m-2'
    mask_t.units = 'degrees_Celsius'
    
    #print 'temp variable:', mask_alex.variables['Tem_Int_0-151']
    
    mask_t[:] =vars()['chl_tot_196_'+str(file[12:16])][:] # putting our df inside our variable
    
    mask_alex.close() # writing the netcdf file!
                               
    #print ' '

moyenne_tem_1970.nc
Creating ncdf
moyenne_tem_1971.nc
Creating ncdf
moyenne_tem_1972.nc
Creating ncdf
moyenne_tem_1973.nc
Creating ncdf
moyenne_tem_1974.nc
Creating ncdf
moyenne_tem_1975.nc
Creating ncdf
moyenne_tem_1976.nc
Creating ncdf
moyenne_tem_1977.nc
Creating ncdf
moyenne_tem_1978.nc
Creating ncdf
moyenne_tem_1979.nc
Creating ncdf
moyenne_tem_1980.nc
Creating ncdf
moyenne_tem_1981.nc
Creating ncdf
moyenne_tem_1982.nc
Creating ncdf
moyenne_tem_1983.nc
Creating ncdf
moyenne_tem_1984.nc
Creating ncdf
moyenne_tem_1985.nc
Creating ncdf
moyenne_tem_1986.nc
Creating ncdf
moyenne_tem_1987.nc
Creating ncdf
moyenne_tem_1988.nc
Creating ncdf
moyenne_tem_1989.nc
Creating ncdf
moyenne_tem_1990.nc
Creating ncdf
moyenne_tem_1991.nc
Creating ncdf
moyenne_tem_1992.nc
Creating ncdf
moyenne_tem_1993.nc
Creating ncdf
moyenne_tem_1994.nc
Creating ncdf
moyenne_tem_1995.nc
Creating ncdf
moyenne_tem_1996.nc
Creating ncdf
moyenne_tem_1997.nc
Creating ncdf
moyenne_tem_1998.nc
Creating ncdf
moyenne_tem_19